In [26]:
import pandas as pd
import numpy as np 
import itertools
import warnings

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from scipy.stats import ttest_ind_from_stats

warnings.simplefilter("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", category=UserWarning)

# Transformando as variáveis categóricas em numéricas. Trocando os atributos da classe ` class` de e -> `0` e p -> `1`

In [13]:
df = pd.read_csv('./dados/agaricus_lepiota_small_c.csv')

# Separando variáveis dependentes e independentes
y = df[['class']].values.ravel()
X = df.drop(['class', 'stalk-root'], axis=1)

# Transformando coluna "class" de e -> 0 e p -> 1
y = np.where(y == 'e', 0, 1)

# Transformando todas as variáveis categóricas em numéricas
ct = ColumnTransformer(
    transformers=[
        ('oe', OrdinalEncoder(), X.columns)
    ],
    remainder='passthrough'
)

# Aplicando a transformação em todas as colunas
X_oe = ct.fit_transform(X)

# Avaliando o desempenho do classificador k-nn com validação cruzada de 2 níveis

In [22]:
# Separando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_oe, y, test_size=0.3, random_state=42)

k1 = 10 #controla o número de vias da validação cruzada para estimar o desempenho do modelo
k2 = 5 #controla o número de vida da validação cruzada para otimização de hiperparametros

# Definindo a métrica personalizada para acurácia da classe positiva (poisonous)
def accuracy_positive_class(y_true, y_pred):
    positive_indices = y_true == 'poisonous'
    return accuracy_score(y_true[positive_indices], y_pred[positive_indices])

#usar o protocolo de validação cruzada estratificada
skf = StratifiedKFold(n_splits=k1, shuffle=True, random_state=1)

acuracias = []

#a função split retorna os índices das instâncias que devem ser usadas para o treinamento e o teste.
for idx_treino, idx_teste in skf.split(X, y):
    
    #extrair as instâncias de treinamento de acordo com os índices fornecidos pelo skf.split
    X_treino = X_oe[idx_treino]
    y_treino = y[idx_treino]
    
    #extrair as instâncias de teste de acordo com os índices fornecidos pelo skf.split
    X_teste = X_oe[idx_teste]
    y_teste = y[idx_teste]
    
    #colocar todas as variáveis na mesma escala, usando o conjunto de treinamento para calcular os parâmetros da escala
    ss = StandardScaler()
    ss.fit(X_treino)
    X_treino = ss.transform(X_treino)
    X_teste = ss.transform(X_teste)
    
    #combinações de parametros otimizar. Aqui estamos apenas otimizando o número de vizinhos mais próximos para o knn (k).
    #Entretanto, podemos colocar todos os valores de todos os parametros. O sklearn se encarrega de gerar todas as combinações.
    params = {'n_neighbors' : range(1,30,2)}
    #instanciar um KNN com parametros padrão
    knn = KNeighborsClassifier()
    #instanciar um GridSearchCV com k2 vias.
    knn = GridSearchCV(knn, params, cv=StratifiedKFold(n_splits=k2), scoring=make_scorer(accuracy_positive_class))
    #realizar a otimização dos hiperparâmetros e treinar o modelo final com a melhor combinação de hiperparametros com todos os dados de treinamento
    knn.fit(X_treino, y_treino)
    
    #calcular a acurácia no conjunto de testes desta iteração e salvar na lista.
    acuracias.append(accuracy_score(y_teste, knn.predict(X_teste)))
    
#calcular as estatísticas da validação cruzada. Estas estatísticas nos dão uma confiança que, na média, este é o desempenho esperado
#do classificador no mundo real.
accs_knn = acuracias
print("min: %.2f, max: %.2f, avg +- std: %.2f+-%.2f" % (min(acuracias), max(acuracias), np.mean(acuracias), np.std(acuracias)))


min: 0.87, max: 0.94, avg +- std: 0.90+-0.02


# Avaliando desempenho do classificador SVM usando validação cruzada em 2 níveis

In [23]:
#Cs e gammas são listas com os valores a serem avaliados para os respectivos parâmetros.
def selecionar_melhor_svm(Cs, gammas, X_treino : np.ndarray, X_val : np.ndarray, 
                          y_treino : np.ndarray, y_val : np.ndarray, n_jobs=4):
    
    def treinar_svm(C, gamma, X_treino, X_val, y_treino, y_val):
        svm = SVC(C=C, gamma=gamma)
        svm.fit(X_treino, y_treino)
        pred = svm.predict(X_val)
        return accuracy_score(y_val, pred)
    
    #gera todas as combinações de parametros C e gamma, de acordo com as listas de valores recebidas por parametro.
    #Na prática faz o produto cartesiano entre Cs e gammas.
    combinacoes_parametros = list(itertools.product(Cs, gammas))
    
    #Treinar modelos com todas as combinações de C e gamma
    acuracias_val = Parallel(n_jobs=n_jobs)(delayed(treinar_svm)
                                       (c, g, X_treino, X_val, y_treino, y_val) for c, g in combinacoes_parametros)       
    
    melhor_val = max(acuracias_val)
    #Encontrar a combinação que levou ao melhor resultado no conjunto de validação
    melhor_comb = combinacoes_parametros[np.argmax(acuracias_val)]   
    melhor_c = melhor_comb[0]
    melhor_gamma = melhor_comb[1]
    
    #Treinar uma SVM com todos os dados de treino e validação usando a melhor combinação de C e gamma.
    svm = SVC(C=melhor_c, gamma=melhor_gamma)
    svm.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])

    return svm, melhor_comb, melhor_val

#Implementa a validação cruzada para avaliar o desempenho da SVM na base de dados com as instâncias X e as saídas y.
#cv_splits indica o número de partições que devem ser criadas.
#Cs é a lista com os valores C que devem ser avaliados na busca exaustiva de parametros para a SVM.
#gammas s é a lista com os valores gamma que devem ser avaliados na busca exaustiva de parametros para a SVM.
def do_cv_svm(X, y, cv_splits, Cs=[1], gammas=['scale']):

    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    acuracias = []
    
    pgb = tqdm(total=cv_splits, desc='Folds avaliados')
    
    for treino_idx, teste_idx in skf.split(X, y):

        X_treino = X[treino_idx]
        y_treino = y[treino_idx]

        X_teste = X[teste_idx]
        y_teste = y[teste_idx]

        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, stratify=y_treino, test_size=0.2, random_state=1)

        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_teste = ss.transform(X_teste)
        X_val = ss.transform(X_val)

        svm, _, _ = selecionar_melhor_svm(Cs, gammas, X_treino, X_val, y_treino, y_val)
        pred = svm.predict(X_teste)

        acuracias.append(accuracy_score(y_teste, pred))
        
        pgb.update(1)
        
    pgb.close()
    
    return acuracias

def calcular_estatisticas(resultados):
    return np.mean(resultados), np.std(resultados), np.min(resultados), np.max(resultados)

def imprimir_estatisticas(resultados):
    media, desvio, mini, maxi = calcular_estatisticas(resultados)
    print("Resultados: %.2f +- %.2f, min: %.2f, max: %.2f" % (media, desvio, mini, maxi))


accs_svm = do_cv_svm(X_oe, y, 10, Cs=[1, 10, 100, 1000], gammas=['scale', 'auto', 2e-2, 2e-3, 2e-4])
imprimir_estatisticas(accs_svm)

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Resultados: 0.90 +- 0.01, min: 0.88, max: 0.91


# Fazendo teste da hipótese nula (pelo Teste-T) para verificar se os resultados obtidos com o KNN e com o SVM são estatisticamente diferentes com 95% de confiança. 

In [28]:
media_knn, std_knn, _, _ = calcular_estatisticas(accs_knn)
media_svm, std_svm, _, _ = calcular_estatisticas(accs_svm)

#calcular o pvalor usando o teste t de Student para duas amostras independentes
_, pvalor = ttest_ind_from_stats(media_knn, std_knn, len(accs_knn), media_svm, std_svm, len(accs_svm))

if pvalor < 0.05:
    print("O p-valor foi %.2f. A diferença entre os classificadores é significativa." % pvalor)
else :
    print("O p-valor foi %.2f. A diferença entre os classificadores não é significativa." % pvalor)

O p-valor foi 0.58. A diferença entre os classificadores não é significativa.


### Você usaria algum classificador que criou para decidir se comeria ou não um cogumelo classificado por ele? Justifique usando o desempenho obtido e o resultado do teste de hipótese. Esta resposta deve estar no final do caderno jupyter, após a análise estatística.

- Considerando que ambos os classificadores tiveram um bom desempenho alcançando até 90% de acurácia, eu ainda teria um pouco de receio em comer algum cogumelo baseado nos resultados dos classificadores. 